# AOC 2022

In [220]:
\cd notebooks

'io



# Advent of Code 2022 - 08

URL: https://adventofcode.com/2022/day/8

We'll study ngn's [solution](https://codeberg.org/ngn/k/src/branch/master/aoc/22/08.k):

This was the program that started my appetite for learning k. I couldn't believe that such an interesting problem could be solved in such a short codebase.

Tbh, at the start of writing this article, I still don't fully understand part2.
The nicest part of this solution is that the solution makes a lot of sense from the intuition point of view.

In [3]:
/ideas from https://codeberg.org/ovs/aoc2022/src/branch/master/08.k
/and https://github.com/chrispsn/aoc2022/blob/main/answers.ngnk
x:0:"i/08"
r:,/1|:\'1+:\x
R:,/{+y}'\{|y}\'2 2#
+//|/R@(-1>':|\)'r                 /part1
|//*/R@{x(+/1|':&\>/:)'1_(1_)\x}'r /part2

1849
201600


## Visibility of a row of trees

Let's start with a minimal example, and create a 1d forest, of 1 line of trees

In [156]:
:t:10?10

1 1 1 2 9 8 4 4 4 5


To find which trees are visible from the left, what we do is a max-scan, and compare each value with the max-so-far. If the current tree is higher, max-so-far is the value of the current tree, and we increase "visibles" by 1. 

Well. that, but in array mode :)

In [196]:
1_>':|\-1,t 

1 0 0 1 1 0 0 0 0 0


A nicer way than prepending -1 and then cutting the first value, is to use the default value for `':` itself

In [197]:
-1>':|\t

1 0 0 1 1 0 0 0 0 0


Now, let's try the same in a little forest

In [124]:
:x:4 4#10?20

(7 13 16 5
 9 14 5 12
 4 18 7 13
 16 5 9 14)


In [151]:
(-1>':|\)x  /add parenthesis to make it a train (

(1 1 1 1
 1 1 0 1
 0 1 0 1
 1 0 0 1)


Little detail worth noting: We use the full forest as a parameter to the train, so that means that all adverbs apply at the row level. We are doing the maxes of one row against the next, against the next... Giving the cumulative max "vertically".

If we'd add a `'`, we would be doing the operations withing each row, so the "max" would be columnwise.

In [ ]:
|\x
|\'x

(7 13 16 5
 9 14 16 12
 9 18 16 13
 16 18 16 14)
(7 13 16 16
 9 14 14 14
 4 18 18 18
 16 16 16 16)


## Rotate and repeat

The approach to solve this exercise is to solve the visibility problem for each one of the trees in the forest, and then, replicate the solution for each side of the forest.

The smart thing arraywise is to rotate the forest, and repeat the function, for each of the 4 orientations that result from rotating 90degrees.

Rotation of a table is quite easy in k. It's just a train of flip and reverse. The order of the operations dictates the way rotation.

In [41]:
:t:3 3#!9
+t /flip
|+t /flip+reverse = rotate
+|t /reverse+flip = rotate

(0 1 2
 3 4 5
 6 7 8)
(0 3 6
 1 4 7
 2 5 8)
(2 5 8
 1 4 7
 0 3 6)
(6 3 0
 7 4 1
 8 5 2)


In [43]:
+$(|+:)\t

(((,"0";,"1";,"2");(,"2";,"5";,"8");(,"8";,"7";,"6");(,"6";,"3";,"0"))
 ((,"3";,"4";,"5");(,"1";,"4";,"7");(,"5";,"4";,"3");(,"7";,"4";,"1"))
 ((,"6";,"7";,"8");(,"0";,"3";,"6");(,"2";,"1";,"0");(,"8";,"5";,"2")))


Now, here's a bit of a different solution from ngn's.

First we scan-rotate 3 times (keeping the original)

```
mv:(-1>':|\)'(+|:)\x
+//|/((!4) (|+:)/'mv)
```

In [202]:
mv:(-1>':|\)'(+|:)\x

We scan rotate `(+|:)` `x`. Remember that `monad scan` without an explicit number of iterations will stop when one iteration result is either the equal to the previous one, or equal to the initial argument. In this case, we know we'll end up with 4 iterations. After the 3rd iteration, the result would be the same as the original one.

A nice trick is `flip` the result to get a visual of how they look like in 2D. Also, `$` will pad the numbers with comma (on the single digit ones), so we can see them aligned.


In [203]:
$x

(("18";,"4";,"6";,"1")
 (,"0";"19";,"9";"18")
 ("17";"10";"18";,"4")
 (,"6";,"1";,"0";"19"))


In [204]:
$+(+|:)\x
`
$+mv

((("18";,"4";,"6";,"1");(,"6";"17";,"0";"18");("19";,"0";,"1";,"6");(,"1";"18";,"4";"19"))
 ((,"0";"19";,"9";"18");(,"1";"10";"19";,"4");(,"4";"18";"10";"17");(,"6";,"9";"18";,"0"))
 (("17";"10";"18";,"4");(,"0";"18";,"9";,"6");("18";,"9";"19";,"0");(,"4";"19";"10";,"1"))
 ((,"6";,"1";,"0";"19");("19";,"4";"18";,"1");(,"1";,"6";,"4";"18");("18";,"0";"17";,"6")))
`
(((,"1";,"1";,"1";,"1");(,"1";,"1";,"1";,"1");(,"1";,"1";,"1";,"1");(,"1";,"1";,"1";,"1"))
 ((,"0";,"1";,"1";,"1");(,"0";,"0";,"1";,"0");(,"0";,"1";,"1";,"1");(,"1";,"0";,"1";,"0"))
 ((,"0";,"0";,"1";,"0");(,"0";,"1";,"0";,"0");(,"0";,"0";,"1";,"0");(,"0";,"1";,"0";,"0"))
 ((,"0";,"0";,"0";,"1");(,"1";,"0";,"0";,"0");(,"0";,"0";,"0";,"1");(,"1";,"0";,"0";,"0")))


This shows the different rotations and the trees that can be seen from the "north".

Next step is to add those numbers up. The intuitive operation is a max reduce over all the forest, but the problem here is that we can't just add the different rotated forests. That would confuse which tree is which. For example, the highest tree in the forest will be visible from all directions, but that doesn't mean we have to count if 4 times but only one.

The nice trick here is to rotate each one of the visibility matrices, a different number of times in reverse direction.

When we created the different rotations, we used +|:, to rotate clockwise. Now we want to rotate counterclockwise `|+:`.

We want to take the 4 matrices and rotate one of them once counterclockwise, the next one, twice counterclockwise, and a third, 3 times. One of them can be left as is (which is the same as rotating 4 times).

So it's a way to unroll the rotations so the same positions refer to the same tree, before we can do the big aggregation.

In [205]:
(!4)(|+:)/'mv

((1 1 1 1;0 1 1 1;0 0 1 0;0 0 0 1)
 (1 0 0 0;1 1 0 0;1 0 1 0;1 0 0 1)
 (1 0 0 0;0 1 0 0;1 1 1 0;1 1 1 1)
 (1 0 1 1;0 1 0 1;0 0 1 1;0 0 0 1))


This is pretty neat: We're nesting an `each` with an `over monad`.

`l m'l`  where `m` is `(|+:)/`. 

For a simplified version of `each+over monad`, look at the following.

## Each+ Monad

In [75]:
1 4 (100+)/'5 8

105 408


1 4 is the left `y` arguments. `5 8` is the right argument `x`. And the operation is a monad `(100+)`, so the `y` argument falls into being the number of repetitions for the monad.

If it would be a dyad, 1 and 4 would be part the `y`s for the dyad:

`1 4 +/'5 8`



In [34]:
1 4+/'5 8

6 12


In [78]:
1 4/'5 8 / and this?? wtf is this one?

25 40


## The `dyad//` pattern

Last cool idiom for this part 1 is `+//`.  In K, some verbs or adverbs have surprising applications when applied repeatedly.

over(`/`) applied to a monad will repeat the application of the monad until it reaches a fixed point. The computation is a monad itsef. Well, there's this pattern where the monad is another `over` application. 

Let's see an example with `,//`. `,/` flattens one list one level.  But if we keep applying flattening until the result doesn't change anymore, we can have a powerful flattening trick that will drill down and flatten all the elements to 1 level, no matter the depth of our list.

In [77]:
,//2 2 2 2#!100

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15


In fact, we can use `scan` instead, to see the intermediate steps

In [73]:
,/\2 2 2 2#!100

((((0 1;2 3);(4 5;6 7));((8 9;10 11);(12 13;14 15)))
 ((0 1;2 3);(4 5;6 7);(8 9;10 11);(12 13;14 15))
 (0 1;2 3;4 5;6 7;8 9;10 11;12 13;14 15)
 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15)


## Back to the challenge

So part 1, in the end, looks like the sum max (`|/`) per tree (meaning, is this tree visible from at least one direction) and summing all those trees that are visible somehow `+//`

In [79]:
+//|/(!4)(|+:)/'mv

14


## Part 2

Part 2 is even crazier. We have to look at the tree visibility not from the outside, but from the inside of the forest, and for each one of the trees.

Intuition here is also "ovbious", once you're in "K mode" (only half joking):
For each rotation, We crack open the forest, and do the visibility function for each one of the trees, then we reassemble the rotations again.

Note that the visibility function here is a bit different than in part 1. For a given tree `t`, we're looking for trees that are shorter than `t`, and that are not behind one that is equal or taller than `t`. Note the difference with part1.

In part1, 
- in a tree sequence like `1 4 3 4`, for `1`, we would see the first `4`, but not the second. 
- in a tree sequence like `1 5 3 4`, for `1` we would see the first `5`, but not the `4`

In part2,
- in a tree sequence like `4 3 2 3`, for `4`, we would see both`3`s (also the `2`)
- in a tree sequence like `4 2 1 3`, for `4`, we would see all `2 1 3`
- in a tree sequence like `4 3 2 1`, for `4`, we would see all `3 2 1`
- in a tree sequence like `4 1 2 3`, for `4`, we would see all `1 2 3`
- in a tree sequence like `4 3 1 2`, for `4`, we would see all `3 1 2`

So valleys count differently here. That's why, in the filtering, there's a kind of cross product comparison (capped by an `over-min`), instead of just a single cumulative `over`.  

In [40]:
|//*/R@{x(+/1|':&\>/:)'1_(1_)\x}'r /part2

201600


We use `(_1)\` to "peel off" the rows of trees one by one. By the challenge statement, the trees on the boundaries are disqualified, so that's why the leading `1_`.

We only see the trees "downwards", but that's going to be solved by the rotation/unrotation pattern.

Now, each one of the rows is "the top row" in some of the peeled intermediate result.

For each one of the "virtual peeled forests", evaluate the train `(+/1|':&\>/:)`, passing as the left parameter, the full forest.
That means that for you compare each one of the trees with each one in the same column in rows below itself. 

- each right (that means, the result has the same size as the number of columns)
- 

In [43]:
:foo:4 4#20?20

(2 2 2 4
 19 17 9 9
 8 11 5 17
 10 13 19 1)


In [196]:
{1_(1_)\x}foo                       /peeling the forest
{x(,/:)'1_(1_)\x}foo                /concat the operands to see what are we comparing against what
{x(>/:)'1_(1_)\x}foo                /is the tree in the intermediate row smaller than the tree in `x`?
{x(&\>/:)'1_(1_)\x}foo              /are all the intermediate ones so far smaller?
{x(1|':&\>/:)'1_(1_)\x}foo          /': no idea
{x(+/1|':&\>/:)'1_(1_)\x}foo        / I thought it was to accomodate the first row, but the rotations should do it     

((19 17 9 9;8 11 5 17;10 13 19 1)
 (8 11 5 17;10 13 19 1)
 ,10 13 19 1
 0#,0N 0N 0N 0N)
((2 2 2 4 19 17 9 9;2 2 2 4 8 11 5 17;2 2 2 4 10 13 19 1)
 (19 17 9 9 8 11 5 17;19 17 9 9 10 13 19 1)
 ,8 11 5 17 10 13 19 1
 ())
((0 0 0 0;0 0 0 0;0 0 0 1)
 (1 1 1 0;1 1 0 1)
 ,0 0 0 1
 0#,0#,0N 0N 0N 0N)
((0 0 0 0;0 0 0 0;0 0 0 0)
 (1 1 1 0;1 1 0 0)
 ,0 0 0 1
 0#,0#,0N 0N 0N 0N)
((1 1 1 1;0 0 0 0;0 0 0 0)
 (1 1 1 1;1 1 1 0)
 ,1 1 1 1
 0#,0#,0N 0N 0N 0N)
(1 1 1 1
 2 2 2 1
 1 1 1 1
 0)


In [121]:
foo>/:1_foo

((0 0 0 0;1 0 1 0;0 0 0 0;0 1 1 0)
 (1 1 1 1;1 0 1 0;1 1 0 1;0 1 1 0)
 (1 1 1 1;0 0 0 0;0 0 0 0;0 1 1 0))


In [109]:
foo< 18 12 19 10

(1 1 1 1
 0 0 0 1
 1 1 1 1
 1 0 0 1)


In [210]:
:x:4 4 4#20?20

((15 10 4 0;4 4 2 17;1 1 5 15;1 10 14 11)
 (8 15 17 3;15 10 4 0;4 4 2 17;1 1 5 15)
 (1 10 14 11;8 15 17 3;15 10 4 0;4 4 2 17)
 (1 1 5 15;1 10 14 11;8 15 17 3;15 10 4 0))


In [211]:
*2 2#x
`s
{|y}\*2 2#x
`s
{|y}\'2 2#x

((15 10 4 0;4 4 2 17;1 1 5 15;1 10 14 11)
 (8 15 17 3;15 10 4 0;4 4 2 17;1 1 5 15))
`s
((15 10 4 0;4 4 2 17;1 1 5 15;1 10 14 11)
 (1 1 5 15;4 4 2 17;15 10 4 0;8 15 17 3))
`s
(((15 10 4 0;4 4 2 17;1 1 5 15;1 10 14 11);(1 1 5 15;4 4 2 17;15 10 4 0;8 15 17 3))
 ((1 10 14 11;8 15 17 3;15 10 4 0;4 4 2 17);(15 10 4 0;8 15 17 3;1 10 14 11;1 1 5 15)))


In [2]:
r:,/1|:\'1+:\x
+$r
R:{+y}'\{|y}\'2 2#
(-1>':|\)'r 
R@(-1>':|\)'r

0#,("";"";"";"")
(()
 ()
 ()
 ())
((();())
 (();()))
